## 청와대 홈페이지 크롤링 코드

### 작성자: 송채은


In [ ]:
from selenium import webdriver    #동적 구현을 위한 셀리늄 활용
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome(#'C:/chromedriver.exe')
driver.get('https://www1.president.go.kr/search')
import time

In [ ]:
search_box = driver.find_element_by_css_selector("li.search_PG_bar input")
search_box.send_keys("우한폐렴")
search_box.send_keys(Keys.RETURN)

In [ ]:
detail_search_button = driver.find_element_by_css_selector("#contents > div.search_PG_contens > div > ul > li:nth-child(6) > a")
detail_search_button.click()

In [ ]:


titles = []
start_date = []
end_date = []
num_consent = []
address = []

for n in range(1,20): #n은 페이지 수    
# get titles, address(url)
    res = driver.page_source
    soup = BeautifulSoup(res, "html.parser")
    contents = soup.find_all('div', class_='PG_contens_pd_list')
    for i in range(0,len(contents)):
        content = contents[i]
        titles.append(content.h2.get_text())
        add_url = str(content.find('a')['href'])
        address.append('https://www1.president.go.kr'+add_url)
    
    # 청원시작과 끝 날짜, 참여인원, 답변일 정보 얻기 
    infos = soup.find_all('ul', class_='PG_peti_list')
    for info in infos: 
        date = info.find_all('span')
        start_date.append(date[0].get_text())
        #print(start_date)
        end_date.append(date[1].get_text())
        #print(end_date)
        num_consent.append(date[2].get_text())
        
    print(n)    
    page_bar = driver.find_elements_by_css_selector('div.p_btn > a')
    try:
        if n%7 != 0:
            page_bar[n%7+1].click()
            time.sleep(1)
        else:
            page_bar[8].click()
            time.sleep(2)
    except:
        print("수집완료")
        break
    time.sleep(0.5)
#driver.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
수집완료


In [ ]:
data = {'제목':titles, '청원시작일':start_date, '청원종료일': end_date,
       '참여인원': num_consent,'URL':address}
df = pd.DataFrame(data)
df.describe()

,제목,청원시작일,청원종료일,참여인원,URL
count,162,162,162,162,162
unique,156,26,26,150,156
top,서비스 종사자 마스크 착용,2020-02-03,2020-03-04,198,https://www1.president.go.kr/petitions/584706
freq,2,58,58,2,2


In [ ]:
df = df.drop_duplicates('제목', keep='first')
df.describe()

,제목,청원시작일,청원종료일,참여인원,URL
count,156,156,156,156,156
unique,156,26,26,150,156
top,우한 교민 720명 잠복기간 14일 동안 크루즈나 섬에서 안전하게 머물게 해주세요.,2020-02-03,2020-03-04,198,https://www1.president.go.kr/petitions/584896
freq,1,55,55,2,1


In [ ]:
df.to_csv('BlueHouse Petition data.csv', sep=',', encoding='utf-8')

In [ ]:
petition = pd.read_csv('BlueHouse Petition data.csv', thousands=',', encoding='utf-8', index_col=0)
petition.head()

,제목,청원시작일,청원종료일,참여인원,URL
0,유튜버 우**을 구속바랍니다,2020-05-09,2020-06-08,1205,https://www1.president.go.kr/petitions/588717
1,우리 아이들 의 생명권 과 자유권 과 교육권 을 묵살한 문재인대통령 은 해명해 주십시요,2020-04-13,2020-05-13,668,https://www1.president.go.kr/petitions/588060
2,재난 기금이라는 말은 재난으로 고통받는 한반도에 거주하는 시민에게 주는 것입니다. ...,2020-04-03,2020-05-03,2747,https://www1.president.go.kr/petitions/587705
3,외국인 입국금지 반드시 해야합니다. 국무총리 당신의 의견이 국민의 의견이라고 생각하...,2020-04-01,2020-05-01,4209,https://www1.president.go.kr/petitions/587596
4,예배로 인한 코로나19 확산 구상권은 대통령에게,2020-03-27,2020-04-26,742,https://www1.president.go.kr/petitions/587312


In [ ]:
df.to_excel('BlueHouse Petition data.xlsx')

In [ ]:
import re 
import time
from tqdm import tqdm_notebook
from urllib.request import urlopen
from bs4 import BeautifulSoup

category =[]
content = []
person = []

for n in tqdm_notebook(petition.index): 
    url = urlopen(petition['URL'][n]) 
    if n / 2 == 0:
        time.sleep(1)
    else: 
        time.sleep(2)
    soup = BeautifulSoup(url, "html.parser") 
    
    # get category 
    sector = soup.find('ul', class_='petitionsView_info_list')
    try: 
        category.append(sector.li.get_text()[4:])
    except AttributeError:
        print('AttributeError: index-', n, 'URL-', petition['URL'][n])
    person.append(sector.find_all('li')[3].get_text()[3:-6])


    # get petition content 
    content_raw = soup.find_all('div', class_='View_write')
    try: 
        if len(content_raw) > 1:
            content_raw_text = content_raw[1].get_text()
        else: 
            content_raw_text = content_raw[0].get_text()   
    except IndexError: 
        print('IndexError:', petition['URL'][n])
    
    content_split = re.split('\r|\t|\n|\xa0', content_raw_text)
    content.append(''.join(content_split))

C:\Users\SONG\anaconda3\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print(len(category), len(content), len(person))

156 156 156


In [ ]:
petition['카테고리'] = category
petition['청원내용'] = content
petition['청원인'] = person

In [ ]:
petition.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 0 to 155
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      156 non-null    object
 1   청원시작일   156 non-null    object
 2   청원종료일   156 non-null    object
 3   참여인원    156 non-null    int64 
 4   URL     156 non-null    object
 5   카테고리    156 non-null    object
 6   청원내용    156 non-null    object
 7   청원인     156 non-null    object
dtypes: int64(1), object(7)
memory usage: 16.0+ KB


In [ ]:
petition.to_csv('Final BlueHouse Petition data.csv', sep=',', encoding='utf-8')